In [1]:
import requests
from datetime import datetime
import time
import json
import glob

import pandas as pd
import numpy as np
import geopandas as gpd
from pytz import timezone
%matplotlib inline
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

In [2]:
api_key = 'ac31e02046794bd3f959fbc98fc25231'

In [3]:
gdf = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
center = gpd.GeoDataFrame(gdf[['GID_2', 'NAME_2']])
center['geometry'] = gdf.centroid
center = center.to_crs(gdf.crs)
center['lat'] = center.geometry.y
center['lon'] = center.geometry.x
center
# gdf

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\2083312229.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center['geometry'] = gdf.centroid
C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\2083312229.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  center['geometry'] = gdf.centroid


,GID_2,NAME_2,geometry,lat,lon
0,SRB.1.1_1,Bor,POINT (22.05425 44.07092),44.070917,22.054249
1,SRB.1.2_1,Kladovo,POINT (22.47141 44.56865),44.568655,22.471406
2,SRB.1.3_1,Majdanpek,POINT (22.058 44.38801),44.388008,22.057996
3,SRB.1.4_1,Negotin,POINT (22.42711 44.24799),44.247986,22.427111
4,SRB.2.1_1,Golubac,POINT (21.74048 44.60889),44.608886,21.740484
...,...,...,...,...,...
156,SRB.25.6_1,Požega,POINT (20.0178 43.88806),43.888056,20.017801
157,SRB.25.7_1,Priboj,POINT (19.40003 43.50832),43.508319,19.400028
158,SRB.25.8_1,Prijepolje,POINT (19.62589 43.32048),43.320476,19.625891
159,SRB.25.9_1,Sjenica,POINT (20.00164 43.20967),43.209667,20.001638


In [4]:
gid_2 = center['GID_2'].to_list()
lat = center['lat'].to_list()
lon = center['lon'].to_list()

loc_dict = {}
for i in range(len(gid_2)):
    loc_dict[gid_2[i]] = [lat[i], lon[i]]

In [5]:
loc_dict

{'SRB.1.1_1': [44.070916840003875, 22.05424872664738],
 'SRB.1.2_1': [44.5686546148656, 22.471405549626134],
 'SRB.1.3_1': [44.38800848123445, 22.057996177042046],
 'SRB.1.4_1': [44.247985541215726, 22.42711139779415],
 'SRB.2.1_1': [44.60888632367149, 21.74048439329081],
 'SRB.2.2_1': [44.45935440783599, 21.714726353653496],
 'SRB.2.3_1': [44.52122465631768, 21.335332314037128],
 'SRB.2.4_1': [44.337986357280606, 21.439651473279095],
 'SRB.2.5_1': [44.63989642646239, 21.190605805795062],
 'SRB.2.6_1': [44.687345803589444, 21.43636807884723],
 'SRB.2.7_1': [44.36787554880919, 21.214356338737765],
 'SRB.2.8_1': [44.19508760637038, 21.735714704002095],
 'SRB.3.1_1': [44.549994492875925, 20.371487647191852],
 'SRB.3.2_1': [44.68998770604238, 20.361058549735947],
 'SRB.3.3_1': [44.634950376453595, 20.652155765778986],
 'SRB.3.4_1': [44.37768730613215, 20.3008821128093],
 'SRB.3.5_1': [44.42604587242706, 20.681950629318763],
 'SRB.3.6_1': [44.79295725163482, 20.36386308456866],
 'SRB.3.7_1'

In [39]:
def create_json(loc:str, start: int, end: int) -> dict:
    lat = loc_dict[loc][0]
    lon = loc_dict[loc][1]
    hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    r = requests.get(hist_url)
    json_out = r.json()
    json_out['name'] = loc

    return json_out

year = 2022
start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

for location_name in tqdm(list(loc_dict.keys())):
    print(location_name)
    json_res = create_json(location_name, start_date, end_date)
    with open(f"D:\Work\WB\LDT\countries\SRB\\raw_data\\air_pollution\municipalities\{location_name}_{year}.json", "w") as outfile: 
            json.dump(json_res, outfile)



C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\901510604.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for location_name in tqdm(list(loc_dict.keys())):


  0%|          | 0/161 [00:00<?, ?it/s]

SRB.1.1_1
SRB.1.2_1
SRB.1.3_1
SRB.1.4_1
SRB.2.1_1
SRB.2.2_1
SRB.2.3_1
SRB.2.4_1
SRB.2.5_1
SRB.2.6_1
SRB.2.7_1
SRB.2.8_1
SRB.3.1_1
SRB.3.2_1
SRB.3.3_1
SRB.3.4_1
SRB.3.5_1
SRB.3.6_1
SRB.3.7_1
SRB.3.8_1
SRB.3.9_1
SRB.3.10_1
SRB.3.11_1
SRB.3.12_1
SRB.3.13_1
SRB.3.14_1
SRB.3.15_1
SRB.3.16_1
SRB.3.17_1
SRB.4.1_1
SRB.4.2_1
SRB.4.3_1
SRB.4.4_1
SRB.4.5_1
SRB.4.6_1
SRB.5.1_1
SRB.5.2_1
SRB.5.3_1
SRB.5.4_1
SRB.5.5_1
SRB.5.6_1
SRB.5.7_1
SRB.5.8_1
SRB.5.9_1
SRB.5.10_1
SRB.5.11_1
SRB.5.12_1
SRB.6.1_1
SRB.6.2_1
SRB.6.3_1
SRB.6.4_1
SRB.6.5_1
SRB.6.6_1
SRB.6.7_1
SRB.6.8_1
SRB.7.1_1
SRB.7.2_1
SRB.7.3_1
SRB.7.4_1
SRB.7.5_1
SRB.7.6_1
SRB.8.1_1
SRB.8.2_1
SRB.8.3_1
SRB.8.4_1
SRB.8.5_1
SRB.8.6_1
SRB.8.7_1
SRB.8.8_1
SRB.9.1_1
SRB.9.2_1
SRB.9.3_1
SRB.9.4_1
SRB.10.1_1
SRB.10.2_1
SRB.10.3_1
SRB.10.4_1
SRB.10.5_1
SRB.10.6_1
SRB.10.7_1
SRB.11.1_1
SRB.11.2_1
SRB.11.3_1
SRB.11.4_1
SRB.11.5_1
SRB.11.6_1
SRB.11.7_1
SRB.12.1_1
SRB.12.2_1
SRB.12.3_1
SRB.12.4_1
SRB.13.1_1
SRB.13.2_1
SRB.13.3_1
SRB.14.1_1
SRB.14.2_1
SRB.14

In [111]:
# gdf = gpd.read_file('countries/SRB/gadm41_SRB_2.json')
# center = gpd.GeoDataFrame(gdf[['GID_2', 'NAME_2']])
# center['geometry'] = gdf.centroid
# center = center.to_crs(gdf.crs)
# center['lat'] = center.geometry.y
# center['lon'] = center.geometry.x

# gid_2 = center['GID_2'].to_list()
# lat = center['lat'].to_list()
# lon = center['lon'].to_list()

# loc_dict = {}
# for i in range(len(gid_2)):
#     loc_dict[gid_2[i]] = [lat[i], lon[i]]

# loc_dict

In [112]:
# def create_json(loc:str, start: int, end: int) -> dict:
#     lat = loc_dict[loc][0]
#     lon = loc_dict[loc][1]
#     hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
#     r = requests.get(hist_url)
#     json_out = r.json()
#     json_out['name'] = loc

#     return json_out

# year = 2023
# start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
# end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

# for location_name in list(loc_dict.keys()):
#     print(location_name)
#     json_res = create_json(location_name, start_date, end_date)
#     with open(f"countries/SRB/air_pollution/municipalities/{location_name}_{year}.json", "w") as outfile: 
#             json.dump(json_res, outfile)




In [46]:
paths = glob.glob('D:\Work\WB\LDT\countries\SRB\\raw_data\\air_pollution\municipalities/*_2022.json')
paths.sort()
len(paths)

161

In [43]:
dfs = []

for path in tqdm(paths):
    # print(path)
    with open(path) as f:
        d = json.load(f)
    i = path.rindex('\\')

    no2 = [x['components']['no2'] for x in d['list']]
    so2 = [x['components']['so2'] for x in d['list']]
    co = [x['components']['co'] for x in d['list']]
    o3 = [x['components']['o3'] for x in d['list']]
    pm10 = [x['components']['pm10'] for x in d['list']]
    pm25 = [x['components']['pm2_5'] for x in d['list']]
    dt = [x['dt'] for x in d['list']]

    df = pd.DataFrame({'datetime_unix': dt,
                    'o3': o3,
                    'pm25': pm25,
                    'pm10': pm10,
                    'co': co,
                    'so2': so2,
                    'no2': no2})
    df['lon'] = d['coord']['lon']
    df['lat'] = d['coord']['lat']
    df['location'] = path[i+1:-10]
    dfs.append(df)

full_df = pd.concat(dfs, axis=0).reset_index(drop=True)
full_df['timestamp'] = pd.to_datetime(full_df['datetime_unix'],unit='s')
full_df['timestamp'] = full_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
full_df = full_df.drop(['datetime_unix'], axis=1)
full_df = full_df[['location', 'lon', 'lat', 'timestamp',
                   'o3', 'pm25', 'pm10', 'co', 'so2', 'no2']]
full_df = full_df.sort_values(by=['location', 'timestamp']).reset_index(drop=True)
full_df

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\1850091446.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(paths):


  0%|          | 0/161 [00:00<?, ?it/s]

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2
0,SRB.1.1_1,22.0542,44.0709,2022-01-01 00:00:00+07:00,56.51,7.27,8.66,236.99,14.54,4.28
1,SRB.1.1_1,22.0542,44.0709,2022-01-01 01:00:00+07:00,57.22,6.78,8.33,240.33,14.78,4.07
2,SRB.1.1_1,22.0542,44.0709,2022-01-01 02:00:00+07:00,57.94,6.34,7.97,236.99,13.35,3.73
3,SRB.1.1_1,22.0542,44.0709,2022-01-01 03:00:00+07:00,57.94,5.92,7.53,236.99,11.09,3.30
4,SRB.1.1_1,22.0542,44.0709,2022-01-01 04:00:00+07:00,59.37,5.42,6.96,230.31,9.42,2.87
...,...,...,...,...,...,...,...,...,...,...
1371035,SRB.9.4_1,20.2648,43.7392,2022-12-30 20:00:00+07:00,57.94,5.09,6.88,223.64,1.01,2.38
1371036,SRB.9.4_1,20.2648,43.7392,2022-12-30 21:00:00+07:00,45.06,6.24,9.45,223.64,0.75,4.84
1371037,SRB.9.4_1,20.2648,43.7392,2022-12-30 22:00:00+07:00,37.91,7.20,11.60,230.31,0.59,7.71
1371038,SRB.9.4_1,20.2648,43.7392,2022-12-30 23:00:00+07:00,33.62,8.27,13.78,236.99,0.51,10.97


In [44]:
full_df['year'] = full_df['timestamp'].dt.year
full_df['date'] = full_df['timestamp'].dt.date
full_df

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2,year,date
0,SRB.1.1_1,22.0542,44.0709,2022-01-01 00:00:00+07:00,56.51,7.27,8.66,236.99,14.54,4.28,2022,2022-01-01
1,SRB.1.1_1,22.0542,44.0709,2022-01-01 01:00:00+07:00,57.22,6.78,8.33,240.33,14.78,4.07,2022,2022-01-01
2,SRB.1.1_1,22.0542,44.0709,2022-01-01 02:00:00+07:00,57.94,6.34,7.97,236.99,13.35,3.73,2022,2022-01-01
3,SRB.1.1_1,22.0542,44.0709,2022-01-01 03:00:00+07:00,57.94,5.92,7.53,236.99,11.09,3.30,2022,2022-01-01
4,SRB.1.1_1,22.0542,44.0709,2022-01-01 04:00:00+07:00,59.37,5.42,6.96,230.31,9.42,2.87,2022,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
1371035,SRB.9.4_1,20.2648,43.7392,2022-12-30 20:00:00+07:00,57.94,5.09,6.88,223.64,1.01,2.38,2022,2022-12-30
1371036,SRB.9.4_1,20.2648,43.7392,2022-12-30 21:00:00+07:00,45.06,6.24,9.45,223.64,0.75,4.84,2022,2022-12-30
1371037,SRB.9.4_1,20.2648,43.7392,2022-12-30 22:00:00+07:00,37.91,7.20,11.60,230.31,0.59,7.71,2022,2022-12-30
1371038,SRB.9.4_1,20.2648,43.7392,2022-12-30 23:00:00+07:00,33.62,8.27,13.78,236.99,0.51,10.97,2022,2022-12-30


In [47]:
full_df = full_df.sort_values(by=['location', 'timestamp'])


def calculate_rolling_avg(group):
    # https://environment.ec.europa.eu/topics/air/air-quality/eu-air-quality-standards_en
    # https://uk-air.defra.gov.uk/air-pollution/faq?question=20#:~:text=The%20maximum%20daily%20running%208,average%20concentrations%20over%20a%20year. 

    group = group.sort_index()
    group['o3_8hr_avg'] = group['o3'].rolling(window=8, min_periods=1).mean()
    # group['co_8hr_avg'] = group['co'].rolling(window=8, min_periods=1).mean()
    return group

full_df = full_df.groupby('location').apply(calculate_rolling_avg).reset_index(drop=True)
o3_daily = full_df.groupby(['location', 'year', 'date']).agg(o3_daily=('o3_8hr_avg', 'max')).reset_index()
o3_yearly = o3_daily.groupby(['location', 'year']).agg(o3=('o3_daily', 'mean')).reset_index()
o3_yearly

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\1765404289.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df = full_df.groupby('location').apply(calculate_rolling_avg).reset_index(drop=True)


,location,year,o3
0,SRB.1.1_1,2022,83.223843
1,SRB.1.2_1,2022,82.699687
2,SRB.1.3_1,2022,81.591422
3,SRB.1.4_1,2022,79.895999
4,SRB.10.1_1,2022,85.749346
...,...,...,...
155,SRB.8.7_1,2022,84.135554
156,SRB.8.8_1,2022,84.628065
157,SRB.9.1_1,2022,79.494700
158,SRB.9.2_1,2022,79.423444


In [48]:
pm25_yearly = full_df.groupby(['location', 'year']).agg(pm25=('pm25', 'mean')).reset_index()
pm25_yearly

,location,year,pm25
0,SRB.1.1_1,2022,8.200826
1,SRB.1.2_1,2022,8.020168
2,SRB.1.3_1,2022,8.813018
3,SRB.1.4_1,2022,8.931743
4,SRB.10.1_1,2022,10.482962
...,...,...,...
155,SRB.8.7_1,2022,10.768622
156,SRB.8.8_1,2022,11.244723
157,SRB.9.1_1,2022,9.239583
158,SRB.9.2_1,2022,10.745974


In [49]:
out_df = pd.merge(pm25_yearly, o3_yearly, on=['location', 'year'], how='inner')
out_df = out_df.rename({'location': 'GID_2'}, axis=1)
out_df

,GID_2,year,pm25,o3
0,SRB.1.1_1,2022,8.200826,83.223843
1,SRB.1.2_1,2022,8.020168,82.699687
2,SRB.1.3_1,2022,8.813018,81.591422
3,SRB.1.4_1,2022,8.931743,79.895999
4,SRB.10.1_1,2022,10.482962,85.749346
...,...,...,...,...
155,SRB.8.7_1,2022,10.768622,84.135554
156,SRB.8.8_1,2022,11.244723,84.628065
157,SRB.9.1_1,2022,9.239583,79.494700
158,SRB.9.2_1,2022,10.745974,79.423444


In [50]:
out_df.to_csv('D:\Work\WB\LDT\countries\SRB\datasets\SRB_air_pollution_2022_GID2.csv', index=False)

In [32]:
# gb = full_df.groupby(['location', 'year']).agg({'pm25': 'mean',
#                                            'pm10': 'mean',
#                                            'no2': 'mean'}).reset_index()
# gb = gb.rename({'location': 'NUTS_ID'}, axis=1)
# gb = gdf[['NUTS_ID', 'NAME_LATN']].merge(gb, on=['NUTS_ID'], how='left')
# gb = gb.rename({'NAME_LATN': 'DISTRICT'}, axis=1)
# gb.columns = gb.columns.str.upper()
# gb['UNIT'] = 'ug/m3'
# gb

In [33]:
# gb.to_csv('countries/SRB/datasets/SRB_air_pollution_2023_NUTS3.csv', index=False)

In [34]:
# gdp = pd.read_csv('countries/SRB/data/gdp/EU_GDP_2021_PPP.csv')
# gdp

In [124]:
# srb_gdp = gdp[(gdp['NUTS_ID'].str.contains('RS')) & (gdp['LEVL_CODE'] == 3)]
# srb_gdp = srb_gdp[srb_gdp['NUTS_ID'] != 'RSZZZ']
# srb_gdp = srb_gdp[srb_gdp['UNIT'] == 'Million EUR']
# srb_gdp = srb_gdp[srb_gdp['YEAR'] == 2023].reset_index(drop=True)
# srb_gdp['VALUE'] = srb_gdp['VALUE']*1000000
# srb_gdp['UNIT'] = 'EUR 2021'
# srb_gdp = pd.merge(srb_gdp, gdf[['NUTS_ID', 'NAME_LATN']], on=['NUTS_ID'], how='left')
# srb_gdp = srb_gdp.rename({'VALUE': 'GDP',
#                           'NAME_LATN': 'DISTRICT'}, axis=1)
# srb_gdp = srb_gdp[['NUTS_ID', 'DISTRICT', 'YEAR', 'GDP', 'UNIT']]
# srb_gdp.head()

,NUTS_ID,DISTRICT,YEAR,GDP,UNIT
0,RS110,City of Belgrade,2023,2.788723e+10,EUR 2021
1,RS121,Zapadnobačka oblast,2023,1.272390e+09,EUR 2021
2,RS122,Južnobanatska oblast,2023,2.453800e+09,EUR 2021
3,RS123,Južnobačka oblast,2023,7.281000e+09,EUR 2021
4,RS124,Severnobanatska oblast,2023,1.055790e+09,EUR 2021


In [35]:
# srb_gdp.to_csv('countries/SRB/datasets/SRB_gdp_2023_NUTS3.csv', index=False)